In [ ]:
import sys, os
sys.path.insert(0, '/data1/home/rocks/discrete_elasticity/')
sys.path.insert(0, '/data1/home/rocks/discrete_elasticity/python_src/')

import numpy as np
import numpy.random as rand
import numpy.linalg as la
import scipy as sp
import gzip
import cPickle as pickle
import itertools as it
import portalocker
import scipy.optimize as sciopt

import mech_network_solver as mns
import network as nw
import tuning_algs as talgs


import matplotlib.pyplot as plt; plt.rcdefaults()
import matplotlib as mpl
import seaborn as sns

%matplotlib inline

sns.set(color_codes=True)
sns.set_context('poster', font_scale=1.75)
sns.set_palette("hls", 8)
sns.set_style('ticks', {'xtick.direction': 'in','ytick.direction': 'in', 'axes.linewidth': 2.0})
mpl.rcParams['mathtext.fontset'] = 'cm'


In [ ]:
NN = 16
eta = 1e-1
NTS = 32
DIM = 2
Lp = -1.0
irec = 0


nw_label = "network_periodic_jammed/network_N{:05d}_Lp{:.2f}/network_irec{:04d}".format(NN, Lp, irec)

with gzip.open("/data1/home/rocks/data/{:}.pklz".format(nw_label), 'rb') as pkl_file:

    nw_data = pickle.load(pkl_file)
    net = nw_data['network']    

NF = 1

edgei = net.edgei
edgej = net.edgej


inodesi = [[] for t in range(NF)]
inodesj = [[] for t in range(NF)]

onodesi = [[] for t in range(NF)]
onodesj = [[] for t in range(NF)]

edge = range(net.NE)

rand.seed(2)

rand.shuffle(edge)

b = edge.pop()

inodesi[0].append(edgei[b])
inodesj[0].append(edgej[b])

for i in range(NTS):
    b = edge.pop()

    onodesi[0].append(edgei[b])
    onodesj[0].append(edgej[b])


print "inodes", inodesi, inodesj

print "onodes", onodesi, onodesj

isvec = [[] for t in range(NF)]
for t in range(NF):
    for (i, j) in zip(inodesi[t], inodesj[t]):
        posi = net.node_pos[DIM*i:DIM*i+DIM]
        posj = net.node_pos[DIM*j:DIM*j+DIM]
        bvec = posj - posi
        bvec -= np.round(bvec/net.L)*net.L
        isvec[t].extend(bvec) 

istrain = [[] for t in range(NF)]
istrain[0].append(1.0)


osvec = [[] for t in range(NF)]
ostrain = [[] for t in range(NF)]
for t in range(NF):
    for (i, j) in zip(onodesi[t], onodesj[t]):
        posi = net.node_pos[DIM*i:DIM*i+DIM]
        posj = net.node_pos[DIM*j:DIM*j+DIM]
        bvec = posj - posi
        bvec -= np.round(bvec/net.L)*net.L
        osvec[t].extend(bvec) 

        r = rand.randint(2)
        ostrain[t].append((2*r-1) * eta)

pert = []
meas = []
for t in range(NF):
    pert.append(talgs.Perturb())
    pert[t].setInputStrain(len(inodesi[t]), inodesi[t], inodesj[t], istrain[t], isvec[t])

    meas.append(talgs.Measure())
    meas[t].setOutputStrain(len(onodesi[t]), onodesi[t], onodesj[t], osvec[t])

obj_func = mns.CyIneqRatioChangeObjFunc(len(np.concatenate(ostrain)), net.NE, 
                                        np.zeros(len(np.concatenate(ostrain)), float), np.concatenate(ostrain))    


K_init = np.ones(net.NE, float) / net.eq_length

cypert = []
for p in pert:
    cypert.append(p.getCyPert())

cymeas = []
for m in meas:
    cymeas.append(m.getCyMeas())

K_init = np.ones(net.NE, float) / net.eq_length



In [ ]:
tuner = talgs.TuneContLin(net, pert, meas, obj_func, K_init, alg='LBFGSB')

data = tuner.tune()

print data

K0 = np.copy(data['K'])

upper = K_init
lower = K_init * 1e3*np.finfo(float).eps

print K0 / upper
print K0 / lower

In [ ]:
quad = lambda x, k0, f0, g, h : h/2.0 * (x**2 - k0**2) + (g - h*k0) * (x - k0) + f0
lin = lambda x, k0, f0, g : g * (x - k0) + f0

K0 = np.copy(data['K'])

upper = K_init
lower = K_init * 1e3*np.finfo(float).eps

solver = tuner.solver

solver.setIntStrengths(K_init)
meas = solver.solveMeas()
obj_func.setRatioInit(meas)

solver.setIntStrengths(K0)
(obj, obj_grad, obj_hess) = obj_func.funcHess(solver)

dK = -la.solve(obj_hess, obj_grad)
# dK = - grad / np.diagonal(hess)
K1 = np.clip(K0+dK, lower, upper)


for b in range(net.NE):
    K = np.copy(K0)
    
    
    k0 = K[b]
    f0 = obj
    
    k1 = K1[b]
    K[b] = K1[b]
    f1 = tuner.func(K)
    
    X =  np.logspace(np.log10(lower[b]), np.log10(upper[b]), 100)
    f = []
    for k in X:
        K[b] = k
        f.append(tuner.func(K))
            
    fig, (ax1, ax2) = plt.subplots(2,1)
    ax1.set_title("$\partial_k F={1:.2E}$    $\partial^2_kF={2:.2E}$".format(b, obj_grad[b], obj_hess[b,b]))
    
    ax1.plot(X, f, 'r-', label="exact")
    ax1.plot([k0], [f0], "rs", label="expansion point")
    ax1.plot([k1], [f1], "mo", label="NR step")

    ax1.plot(X, quad(X, k0, f0, obj_grad[b], obj_hess[b,b]), 'g:', label="quadratic")
    
    ax1.plot(X, lin(X, k0, f0, obj_grad[b]), 'b--', label="linear")
    
    
    ax1.set_yscale('log')
    ax1.set_xscale('log')
    ax1.set_xlabel("$k_{{{}}}$".format(b))
    ax1.set_ylabel("$F(k_{{{}}})$".format(b))
    ax1.set_xlim(np.min(X), np.max(X))
    
#     ax1.legend()
    
    
    X =  np.linspace(np.max([0.9*k0, lower[b]]), np.min([1.1*k0, upper[b]]), 100)
    f = []
    for k in X:
        K[b] = k
        f.append(tuner.func(K))
    
    ax2.plot(X, f, 'r-', label="exact")
    ax2.plot([k0], [f0], "rs", label="expansion point")
#     ax2.plot([k1], [f1], "mo", label="NR step")

    ax2.plot(X, quad(X, k0, f0, obj_grad[b], obj_hess[b,b]), 'g:', label="quadratic")
    
    ax2.plot(X, lin(X, k0, f0, obj_grad[b]), 'b--', label="linear")
    
    
#     ax1.set_yscale('log')
#     ax2.set_xscale('log')
    ax2.set_xlabel("$k_{{{}}}$".format(b))
    ax2.set_ylabel("$F(k_{{{}}})$".format(b))
    ax2.set_xlim(np.min(X), np.max(X))
    
#     ax2.legend()
    
    plt.show()

In [ ]:
quad = lambda x, k0, f0, g, h : h/2.0 * (x**2 - k0**2) + (g - h*k0) * (x - k0) + f0
lin = lambda x, k0, f0, g : g * (x - k0) + f0

K0 = np.copy(data['K'])

upper = K_init
lower = K_init * 1e3*np.finfo(float).eps

solver = tuner.solver

solver.setIntStrengths(K_init)
meas = solver.solveMeas()
obj_func.setRatioInit(meas)

solver.setIntStrengths(K0)
(obj, obj_grad, obj_hess) = obj_func.funcHess(solver)

active = [0, 0, 0, 1, 0, 1  1  0  0  0 1  1 -1  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  1  0  0  0  1  0]

print obj

(evals, evecs) = la.eigh(obj_hess)


# dK = -la.solve(obj_hess, obj_grad)
# # dK = - grad / np.diagonal(hess)
# K1 = np.clip(K0+dK, lower, upper)


for b in range(1, net.NE):
    K = np.copy(K0)
    
    
    k0 = K[b]
    f0 = obj
    
    dK = evecs[:, b] / np.max(np.abs(evecs[:, b]))
    
    X =  np.linspace(-1e-3, 1e-3, 1001)
    f = []
    n = []
    for x in X:
        
        n.append(np.sum((K+x*dK) < lower) + np.sum((K+x*dK) > upper))
        
        solver.setIntStrengths(np.clip(K+x*dK, lower, upper))       
        f.append(obj_func.func(solver))
                           
    gdotv = obj_grad.dot(evecs[:, b])
    lam = evals[b]
            
    fig, (ax1, ax2) = plt.subplots(2,1)
    ax1.set_title(r"$\lambda ={1:.2E}$    $\vec{{v}} \cdot \nabla F={2:.2E}$".format(b, lam, gdotv))
    
    ax1.plot(X, f, 'r-', label="exact")
    ax1.plot([0], [f0], "rs", label="expansion point")
#     ax1.plot(X, f0 + X*gdotv + X**2 * lam, 'g:', label="quadratic")
# #     ax1.plot(X, quad(X, k0, f0, obj_grad[b], obj_hess[b,b]), 'g:', label="quadratic")
    
# #     ax1.plot(X, lin(X, k0, f0, obj_grad[b]), 'b--', label="linear")
    
# #     ax1.set_ylim(0.9999 * f0, 1.0001*f0)
    ax1.set_yscale('log')
# #     ax1.set_xscale('log')
# #     ax1.set_xlabel("$k_{{{}}}$".format(b))
#     ax1.set_ylabel("$F(K)$".format(b))
#     ax1.set_xlim(np.min(X), np.max(X))

# #     ax1.legend()

    ax3 = ax1.twinx()
    ax3.plot(X, n, 'b-')
    
    X =  np.linspace(-0.2e-6, 0.2e-6, 1001)
    f = []
    n = []
    for x in X:
        n.append(np.sum((K+x*dK) < lower) + np.sum((K+x*dK) > upper))
        solver.setIntStrengths(np.clip(K+x*dK, lower, upper))       
        f.append(obj_func.func(solver))
                
    ax2.plot(X, f, 'r-', label="exact")
#     ax2.plot([0], [f0], "r.", label="expansion point")

    ax2.plot(X, f0 + X*gdotv + X**2 * lam, 'g:', label="quadratic")
# #     ax1.plot(X, quad(X, k0, f0, obj_grad[b], obj_hess[b,b]), 'g:', label="quadratic")
    
# #     ax1.plot(X, lin(X, k0, f0, obj_grad[b]), 'b--', label="linear")
    
    ax2.set_ylim(0.9999999 * f0, 1.0000001*f0)
    ax2.set_yscale('log')
# #     ax1.set_xscale('log')
# #     ax1.set_xlabel("$k_{{{}}}$".format(b))
#     ax2.set_ylabel("$F(K)$".format(b))
#     ax2.set_xlim(np.min(X), np.max(X))
    
# #     ax1.legend()

    ax4 = ax2.twinx()
    ax4.plot(X, n, 'b-')
    
    plt.show()
    
#     break

In [ ]:
solver = tuner.solver

solver.setIntStrengths(K_init)
meas = solver.solveMeas()
obj_func.setRatioInit(meas)

solver.setIntStrengths(K0)
(obj, obj_grad, obj_hess) = obj_func.funcHess(solver)
# print obj_hess

(evals, evecs) = la.eigh(obj_hess)

obj_approx = np.zeros([len(obj_hess), len(obj_hess)], float)

for i in range(len(evals)):
    obj_approx += evals[i] * np.outer(evecs[:, i], evecs[:, i])
    
print obj_approx - obj_hess

In [ ]:
K = np.copy(data['K'])
# K = K_init



solver = mns.CyLinSolver(net.getCyNetwork(), 1, cypert, cymeas)

solver.setIntStrengths(K)

(obj, obj_grad, obj_hess) = obj_func.funcHess(solver)

# print sorted(np.abs(obj_hess.flatten()))


ax = sns.heatmap(np.log10(np.abs(obj_hess)))
plt.show()

ax = sns.heatmap(obj_hess < 0)

print np.diagonal(obj_hess)

plt.show()

fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
    
sns.distplot(np.log10(obj_hess.flatten()[np.where(obj_hess.flatten() > 0.0)]), kde=False, ax=ax1)
sns.distplot(np.log10(np.abs(obj_hess.flatten()[np.where(obj_hess.flatten() < 0.0)])), kde=False, ax=ax2)
# ax1.set_xscale('log')
# ax1.set_yscale('log')

# ax2.set_xscale('log')
# ax2.set_yscale('log')
plt.show()

In [ ]:
(evals, evecs) = la.eigh(obj_hess)

imin = np.argmin(evals)

print evals[imin]
print evecs[:, imin]

# print sorted(evals)

ineg = np.where(evals < np.finfo(float).eps)[0]

print len(ineg), len(evals), np.min(evals), np.max(evals)

upper = K_init
lower = K_init * 1e3*np.finfo(float).eps

print K / upper
print K/ lower

# eps = 1e4*np.finfo(float).eps

# Hinv = np.zeros([len(evals), len(evals)], float)
# # for i in range(len(evals)):
# #     Hinv += 1.0 / np.max([eps, evals[i]]) * np.outer(evecs[:, i], evecs[:, i])

# for i in range(len(evals)):
#     if evals[i] > eps:
#         Hinv += 1.0 / evals[i] * np.outer(evecs[:, i], evecs[:, i])
    
# print Hinv
        
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
    
sns.distplot(np.log10(evals[np.where(evals > 0.0)]), kde=False, ax=ax1)
sns.distplot(np.log10(np.abs(evals[np.where(evals < 0.0)])), kde=False, ax=ax2)
# ax1.set_xscale('log')
# ax1.set_yscale('log')

# ax2.set_xscale('log')
# ax2.set_yscale('log')
plt.show()

# (evals, evecs) = la.eigh(Hinv)
# print sorted(evals)

In [ ]:
dx = -la.solve(obj_hess, obj_grad)
# dx = -Hinv.dot(obj_grad)

# print sorted(dx)

# print la.solve(obj_hess, obj_grad)
upper = K_init
lower = K_init * 1e3*np.finfo(float).eps

print la.norm(np.clip(K+dx, lower, upper) - K, ord=np.inf)

nz = np.nonzero(K+dx - np.clip(K+dx, lower, upper))
print K[nz]
print dx[nz]
print obj_grad[nz]
print obj_hess[nz, nz]

# for i in range(len(K)):
#     print K[i], K[i] + dx[i], (K[i] + dx[i]) / (K_init[i]* 1e3*np.finfo(float).eps) , (K[i] + dx[i]) /  K_init[i]

In [ ]:
solver.setIntStrengths(K)
obj = obj_func.func(solver)
print obj

solver.setIntStrengths(K+dx)

obj = obj_func.func(solver)
print obj

solver.setIntStrengths(np.clip(K+dx, lower, upper))

obj = obj_func.func(solver)
print obj

In [ ]:
# K = np.copy(data['K'])
K = np.copy(K_init)

solver = mns.CyLinSolver(net.getCyNetwork(), 1, cypert, cymeas)

solver.setIntStrengths(K)

(obj, obj_grad, obj_hess) = obj_func.funcHess(solver)

print obj

obj_hess_approx = np.zeros(net.NE, float)
obj_grad_approx = np.zeros(net.NE, float)

dk = 1e-6

for i in range(net.NE):
    K_tmp = np.copy(K)
    K_tmp[i] -= 0.5 * dk
    
    solver.setIntStrengths(K_tmp)

    obj1 = obj_func.func(solver)
    
    K_tmp = np.copy(K)
    K_tmp[i] += 0.5 * dk
    
    solver.setIntStrengths(K_tmp)

    obj2 = obj_func.func(solver)
    
    obj_grad_approx[i] = (obj2 - obj1) / dk
        
    obj_hess_approx[i] = (obj2 - 2.0 * obj + obj1) / (0.5*dk)**2

In [ ]:
print obj_grad_approx

print obj_grad

# print (obj_grad  - obj_grad_approx) / obj_grad

print obj_hess_approx

print np.diagonal(obj_hess)

# print (obj_hess_approx - np.diagonal(obj_hess)) / np.diagonal(obj_hess)

In [ ]:
def golden_search(x0, p):    
     
    bound = np.sign(p)
        
    u = np.where(bound==1.0)[0]
    l = np.where(bound==-1.0)[0]
    m = np.where(bound==0.0)[0]

    alpha_bound = np.zeros(len(x0), float)
    alpha_bound[u] = (upper[u] - x0[u]) / p[u]
    alpha_bound[l] = (lower[l] - x0[l]) / p[l]
    alpha_bound[m] = np.inf

#     print bound
    
#     print alpha_bound

    imin = np.argmin(alpha_bound)
    alpha_max = alpha_bound[imin]
    
#     print imin, alpha_max, x0[imin]    
    
    phi = (np.sqrt(5.0) + 1.0) / 2.0
            
    a = 0
    solver.setIntStrengths(x0)
    fa = obj_func.func(solver)
    
    f0 = fa
    
    b = alpha_max
    solver.setIntStrengths(x0+b*p)
    fb = obj_func.func(solver)
    
#     print fa, fb, a, b
    sys.stdout.flush()    
    c = b - (b - a) / phi
    d = a + (b - a) / phi 
        
    while np.abs(c - d) > np.finfo(float).eps:
        solver.setIntStrengths(x0+c*p)
        fc = obj_func.func(solver)
        solver.setIntStrengths(x0+d*p)
        fd = obj_func.func(solver)

#         print "Section:", a, c, d, b
#         print "Delta Function:", fa, fc, fd, fb

        if fc <= fd or (fc > fa and fd > fa):
            b = d
            fb = fd
        else:
            a = c
            fa = fc

        c = b - (b - a) / phi
        d = a + (b - a) / phi       
    
    alpha = b
    
    if np.abs(alpha - alpha_max) < np.finfo(float).eps:
        alpha = alpha_max
        return (alpha, [imin], [np.sign(p[imin])])
    else:
        return (alpha, [], [])

    
solver = tuner.solver    
        
upper = K_init
lower = K_init * 1e3*np.finfo(float).eps

K = np.copy(data['K'])
active_mask = np.ones(len(K), bool)
bound = np.zeros(len(K), int)

for i in range(10000):
    
    solver.setIntStrengths(K)
    
    if i % 100 == 0:
    
        (obj, obj_grad, obj_hess) = obj_func.funcHess(solver)

        phess = obj_hess[active_mask, :][:, active_mask]

        evals = la.eigvalsh(phess)
        
        print i, len(evals)
        print "Objective Function:", obj
        print "Evals:", np.max(evals), np.min(evals)
        print "PGrad Max:", np.max(np.abs(obj_grad[active_mask])), la.norm(obj_grad[active_mask])
    
        evals = tuner.evals(K)
        print evals[0:4]
    
    
    else:
        (obj, obj_grad) = obj_func.funcGrad(solver)
#     fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
    
#     sns.distplot(np.log10(evals[np.where(evals > 0.0)]), kde=False, ax=ax1)
#     sns.distplot(np.log10(np.abs(evals[np.where(evals < 0.0)])), kde=False, ax=ax2)
#     # ax1.set_xscale('log')
#     # ax1.set_yscale('log')

#     # ax2.set_xscale('log')
#     # ax2.set_yscale('log')
#     plt.show()
        
    uactive = np.where((bound==1.0) & (obj_grad>0.0))[0]
    lactive = np.where((bound==-1.0) & (obj_grad<0.0))[0]
    
    active_mask[uactive] = True
    bound[uactive] = 0
    active_mask[lactive] = True
    bound[lactive] = 0
    
    p = -obj_grad
        
    p[~active_mask] = 0.0
        
    (alpha, remove, rbound) = golden_search(K, p)
    
    if len(remove) > 0:
        print remove
    
#     print alpha, remove, rbound
    
    K += alpha*p
    
    active_mask[remove] = False
    bound[remove] = rbound
    
#     print "Inactive Set:", np.arange(len(K))[~active_mask]
#     print "Inactive Set Bound:", bound[~active_mask]
        
        

In [ ]:
print data['K'] / upper
print data['K'] / lower

In [ ]:
mask = np.ones(10, dtype=bool)
mask[[0,2,4]] = False

x = np.arange(10)
print x
print x[mask]
print x[~mask]

mask = np.ones(3, dtype=bool)
mask[[1]] = False

mask2d = np.ones([3,3], dtype=bool)
mask2d[~mask, :] = False
mask2d[:, ~mask] = False
print mask2d

x = np.arange(9).reshape(3,3)
print x
print x[mask, :][:, mask]

In [ ]:
func = lambda x, A, B, C: np.log10(A + B*x**C)

x = 1.0/np.array([0.1, 0.05, 0.025, 0.0125, 0.00625, 0.003125, 0.0015625, 0.00078125, 0.000390625, 0.0001953125, 9.765625e-05, 4.8828125e-05, 2.44140625e-05, 1.220703125e-05, 6.103515625e-06, 3.0517578125e-06, 1.52587890625e-06, 7.62939453125e-07, 3.814697265625e-07, 1.9073486328125e-07, 9.5367431640625e-08, 4.76837158203125e-08, 2.384185791015625e-08, 1.1920928955078126e-08, 5.960464477539063e-09, 2.9802322387695314e-09, 1.4901161193847657e-09, 7.450580596923829e-10])
y = np.array([2.365803790320861, 1.6829650332492414, 1.1414304616454087, 0.8118669392917126, 0.5643994525116612, 0.5643994525131625, 0.5643994525131625, 0.5434933152472824, 0.5204507823898046, 0.3629692702888317, 0.30010170544918646, 0.29989247078027154, 0.29980544093165357, 0.2773588560627318, 0.26986544947640384, 0.24025051182094476, 0.19373263323908438, 0.19370209012915499, 0.17772994252720634, 0.17453334359716827, 0.16976603288188902, 0.06626796637733322, 3.230578555238742e-07, 6.533733011639023e-08, 4.5098242034714677e-13, 4.5098243106974683e-13, 3.916159284238644e-13, 9.904243774804398e-22])

fig, ax = plt.subplots(1,1)
ax.plot(x, y, '.')


popt, pcov = sciopt.curve_fit(func, x, np.log10(y), p0=[4.05568125e-03, 3.10127298e+01, -1.34076212e+00], bounds=([0, 0, -np.inf], [np.inf, np.inf, 0]))

print popt


ax.plot(np.logspace(1, 7, 100), 10**func(np.logspace(1, 7, 100), *popt), '-')

ax.set_xscale('log')
ax.set_yscale('log')



plt.show()